<a href="https://colab.research.google.com/github/Lineage-Reconstruction-DREAM-Challenge/C2_C3_training_data_performance/blob/master/FT2_C3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Evaluating the running time of FT2 on a dataset of 2,500 cells with 1,000 targets

In [0]:
!rm -f FastTree.c
!wget http://www.microbesonline.org/fasttree/FastTree.c
!gcc -DNO_SSE -O3 -finline-functions -funroll-loops -Wall -o FastTree FastTree.c -lm

--2020-06-07 22:35:44--  http://www.microbesonline.org/fasttree/FastTree.c
Resolving www.microbesonline.org (www.microbesonline.org)... 128.32.248.24
Connecting to www.microbesonline.org (www.microbesonline.org)|128.32.248.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395543 (386K) [text/plain]
Saving to: ‘FastTree.c’

FastTree.c          100%[===================>] 386.27K   966KB/s    in 0.4s    

2020-06-07 22:35:45 (966 KB/s) - ‘FastTree.c’ saved [395543/395543]



In [0]:
!wget -O C3_test.tsv https://s3.msi.umn.edu/gongx030/rlib/C2_C3_training_data_performance/C3_test.tsv
!wget -O unified_transition_table_for_fastree.mat https://s3.msi.umn.edu/gongx030/rlib/C2_C3_training_data_performance/dream_examples/unified_transition_table_for_fastree.mat

--2020-06-07 22:35:54--  https://s3.msi.umn.edu/gongx030/rlib/C2_C3_training_data_performance/C3_test.tsv
Resolving s3.msi.umn.edu (s3.msi.umn.edu)... 128.101.189.144
Connecting to s3.msi.umn.edu (s3.msi.umn.edu)|128.101.189.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20214893 (19M) [text/plain]
Saving to: ‘C3_test.tsv’

C3_test.tsv         100%[===================>]  19.28M  71.8MB/s    in 0.3s    

2020-06-07 22:35:55 (71.8 MB/s) - ‘C3_test.tsv’ saved [20214893/20214893]

--2020-06-07 22:35:55--  https://s3.msi.umn.edu/gongx030/rlib/C2_C3_training_data_performance/dream_examples/unified_transition_table_for_fastree.mat
Resolving s3.msi.umn.edu (s3.msi.umn.edu)... 128.101.189.144
Connecting to s3.msi.umn.edu (s3.msi.umn.edu)|128.101.189.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4802 (4.7K) [text/plain]
Saving to: ‘unified_transition_table_for_fastree.mat’

unified_transition_ 100%[===================>]   4.69K  --.-

In [0]:
!pip install git+https://github.com/jeetsukumaran/DendroPy.git

  Cloning https://github.com/jeetsukumaran/DendroPy.git to /tmp/pip-req-build-c3oe1gh5
  Running command git clone -q https://github.com/jeetsukumaran/DendroPy.git /tmp/pip-req-build-c3oe1gh5
  Created wheel for DendroPy: filename=DendroPy-4.4.0-cp36-none-any.whl size=450883 sha256=298e07581b97d66d912a6ab2e36ec0b3756deff3462afe91252a90e7d8eeb2b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-hihk3boh/wheels/be/75/21/686ec0b11dc2e8e4824cd5d73ba2f71e438d93decee8b6f1b4
Successfully built DendroPy


In [0]:
import os
import sys
import pandas as pd
import dendropy
import subprocess
import time
import numpy as np

In [0]:
df = pd.read_table('C3_test.tsv', sep = '\t')
df = df[0:2500]

In [0]:
symbol_map = {
    0.0:'A',
    '0':'A',
    'A':'R',
    'B':'N',
    'C':'D',
    'D':'C',
    'E':'Q',
    'F':'E',
    'G':'G',
    'H':'H',
    'I':'I',
    'J':'L',
    'K':'K',
    'L':'M',
    'M':'F',
    'N':'P',
    'O':'S',
    'P':'T',
    'Q':'W',
    'R':'Y',
    'S':'-', # Skipping rare characters as we're running out of standard amino acids
    'T':'-',
    'U':'-',
    'V':'-',
    '-':'V', # Inherited deletion is treated as charcter as it carries information (unlike acquisition dropout)
}

In [0]:
for col in df.columns:
    df[col] = df[col].map(symbol_map)
df.fillna("-",inplace=True)
normalized_newick_path = 'input.fasta'
with open(normalized_newick_path, 'w') as f:
    for sr, row in df.iterrows():
        f.write('>{}\r\n{}\r\n'.format(sr, ''.join(row.values)))

In [0]:
fasttree_path = "./FastTree"
fasttree_transition_table = 'unified_transition_table_for_fastree.mat'

def run_fasttree(newick_output_file, mutations_fasta_path):
    subprocess.call([fasttree_path, 
                 '-pseudo', 
                 '-trans', fasttree_transition_table,
                 '-out', newick_output_file, 
                 mutations_fasta_path])
    good_tree = dendropy.Tree.get_from_path(
                        newick_output_file,
                    "newick")
    return good_tree

In [0]:
newick_output_file = 'FT2.newick'
t0 = time.time()
rec_tree = run_fasttree(newick_output_file, normalized_newick_path)
print('Time elasped: ' + str(np.round(time.time() - t0, 3)) + " seconds")

Time elasped: 366.367 seconds
